In [2]:
import pandas as pd

In [3]:
# Reload the data with semicolon as the delimiter to properly parse the file
options_data = pd.read_csv('OptionsAmerican.csv', delimiter=';')

# Display the first few rows again to check the data
options_data.head()

,Name Underlying,ISIN ubnderlying,Link,Volatility,Risk free rate,Underlying Price,Settle C,Strike Price,Settle P,In Years,N
0,ASML Holding,NL0010273215,https://live.euronext.com/en/product/stock-opt...,"0,3892","0,03288","716,7","238,18","480,00","0,14","0,077",30
1,ASML Holding,NL0010273216,https://live.euronext.com/en/product/stock-opt...,"0,3892","0,03288","716,7","218,37","500,00","0,30","0,077",30
2,ASML Holding,NL0010273217,https://live.euronext.com/en/product/stock-opt...,"0,3892","0,03288","716,7","208,50","510,00","0,43","0,077",30
3,ASML Holding,NL0010273218,https://live.euronext.com/en/product/stock-opt...,"0,3892","0,03288","716,7","198,68","520,00","0,61","0,077",30
4,ASML Holding,NL0010273219,https://live.euronext.com/en/product/stock-opt...,"0,3892","0,03288","716,7","179,16","540,00","1,10","0,077",30


In [4]:
# Replace commas with dots for proper numeric conversion
columns_to_clean = ['Volatility', 'Risk free rate', 'Underlying Price', 'Settle C', 'Strike Price', 'Settle P', 'In Years']

for col in columns_to_clean:
    options_data[col] = options_data[col].str.replace(',', '.').astype(float)

# Display the cleaned data to confirm
options_data.head()

,Name Underlying,ISIN ubnderlying,Link,Volatility,Risk free rate,Underlying Price,Settle C,Strike Price,Settle P,In Years,N
0,ASML Holding,NL0010273215,https://live.euronext.com/en/product/stock-opt...,0.3892,0.03288,716.7,238.18,480.0,0.14,0.077,30
1,ASML Holding,NL0010273216,https://live.euronext.com/en/product/stock-opt...,0.3892,0.03288,716.7,218.37,500.0,0.30,0.077,30
2,ASML Holding,NL0010273217,https://live.euronext.com/en/product/stock-opt...,0.3892,0.03288,716.7,208.50,510.0,0.43,0.077,30
3,ASML Holding,NL0010273218,https://live.euronext.com/en/product/stock-opt...,0.3892,0.03288,716.7,198.68,520.0,0.61,0.077,30
4,ASML Holding,NL0010273219,https://live.euronext.com/en/product/stock-opt...,0.3892,0.03288,716.7,179.16,540.0,1.10,0.077,30


In [6]:
import numpy as np

# Binomial model for option pricing
def binomial_option_pricing(S, K, T, r, sigma, N, option_type="call"):
    # Calculate time step
    dt = T / N
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    
    # Calculate risk-neutral probability
    q = (np.exp(r * dt) - d) / (u - d)
    
    # Stock price tree
    stock_prices = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            stock_prices[j, i] = S * (u ** (i - j)) * (d ** j)
    
    # Option values at maturity
    option_values = np.zeros((N + 1, N + 1))
    if option_type == "call":
        option_values[:, N] = np.maximum(stock_prices[:, N] - K, 0)
    elif option_type == "put":
        option_values[:, N] = np.maximum(K - stock_prices[:, N], 0)
    
    # Backward induction to value the option
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_values[j, i] = np.exp(-r * dt) * (q * option_values[j, i + 1] + (1 - q) * option_values[j + 1, i + 1])
    
    return option_values[0, 0]

# Apply the model for each row in the dataset
calculated_call_prices = []
calculated_put_prices = []

for index, row in options_data.iterrows():
    S = row['Underlying Price']
    K = row['Strike Price']
    T = row['In Years']
    r = row['Risk free rate']
    sigma = row['Volatility']
    N = int(row['N'])
    
    # Calculate call and put prices using binomial model
    call_price = binomial_option_pricing(S, K, T, r, sigma, N, option_type="call")
    put_price = binomial_option_pricing(S, K, T, r, sigma, N, option_type="put")
    
    calculated_call_prices.append(call_price)
    calculated_put_prices.append(put_price)

# Store the calculated prices in new columns
options_data['Calculated Call Price'] = calculated_call_prices
options_data['Calculated Put Price'] = calculated_put_prices

# Calculate deviation between settled and calculated prices
options_data['Call Price Deviation'] = options_data['Settle C'] - options_data['Calculated Call Price']
options_data['Put Price Deviation'] = options_data['Settle P'] - options_data['Calculated Put Price']

# Display the updated data with new columns
options_data[['Settle C', 'Calculated Call Price', 'Call Price Deviation', 'Settle P', 'Calculated Put Price', 'Put Price Deviation']].head()


,Settle C,Calculated Call Price,Call Price Deviation,Settle P,Calculated Put Price,Put Price Deviation
0,238.18,237.914325,0.265675,0.14,0.000618,0.139382
1,218.37,217.968063,0.401937,0.30,0.003784,0.296216
2,208.50,207.999692,0.500308,0.43,0.010127,0.419873
3,198.68,198.032888,0.647112,0.61,0.018038,0.591962
4,179.16,178.134858,1.025142,1.10,0.069437,1.030563


In [11]:
options_data[['Settle C', 'Calculated Call Price', 'Call Price Deviation', 'Settle P', 'Calculated Put Price', 'Put Price Deviation']].tail()

,Settle C,Calculated Call Price,Call Price Deviation,Settle P,Calculated Put Price,Put Price Deviation
1300,1.18,1.964350,-0.784350,2.77,2.665584,0.104416
1301,0.77,1.503340,-0.733340,3.70,3.499254,0.200746
1302,0.63,1.331351,-0.701351,4.21,3.964945,0.245055
1303,0.32,0.876566,-0.556566,5.89,5.442518,0.447482
1304,0.17,0.560229,-0.390229,7.74,7.058541,0.681459


In [8]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [9]:
options_data.to_excel('options_data.xlsx', index=False)